In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\wine.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
X = df.drop(columns=['Customer_Segment']).values
y = df['Customer_Segment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=3)

In [ ]:
tfd = tfp.distributions

def create_bnn(hidden_units=16, learning_rate=0.01):
    model = tf.keras.Sequential([
        tfp.layers.DenseVariational(
            units=hidden_units,
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([hidden_units])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([hidden_units])))
            ),
            activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=hidden_units, 
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([hidden_units])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([hidden_units])))
            ),
            activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=3, 
            make_prior_fn=lambda: tfd.Normal(loc=0., scale=1.),
            make_posterior_fn=lambda: tfd.Normal(
                loc=tf.Variable(tf.random.normal([3])),
                scale=tf.nn.softplus(tf.Variable(tf.random.normal([3])))
            ),
            activation="softmax"
        )
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [ ]:
def objective(trial):
    hidden_units = trial.suggest_int("hidden_units", 8, 64)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    model = create_bnn(hidden_units, learning_rate)
    
    history = model.fit(X_train, y_train_onehot, epochs=50, verbose=0, batch_size=16, validation_split=0.2)
    
    val_loss = history.history["val_loss"][-1]
    return val_loss

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

In [ ]:
best_params = study.best_params
print("Best Hyperparameters:", best_params)

In [ ]:
best_model = create_bnn(hidden_units=best_params["hidden_units"], learning_rate=best_params["learning_rate"])

history = best_model.fit(X_train, y_train_onehot, epochs=100, batch_size=16, verbose=1, validation_split=0.2)

In [ ]:
y_pred_probs = np.array([best_model(X_test) for _ in range(100)])
y_pred_mean = y_pred_probs.mean(axis=0) 
y_pred_std = y_pred_probs.std(axis=0) 

In [ ]:
y_pred_classes = np.argmax(y_pred_mean, axis=1)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_classes)
loss = log_loss(y_test_onehot, y_pred_mean)

print(f"Accuracy: {accuracy:.4f}")
print(f"Log Loss: {loss:.4f}")

In [ ]:
plt.figure(figsize=(10, 5))
for i in range(3):
    plt.hist(y_pred_probs[:, :, i].flatten(), bins=50, alpha=0.5, label=f"Kelas {i}")

plt.title("Distribusi Probabilitas Prediksi Bayesian")
plt.xlabel("Probabilitas Prediksi")
plt.ylabel("Frekuensi")
plt.legend()
plt.show()
